# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn.preprocessing import normalize
import os

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, np.array([vertices], dtype=np.int32), ignore_mask_color)
    #plt.figure()
    #plt.imshow(weighted_img(mask, img, β=0.3))
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns a list of lines of (x1,y1,x2,y2).
    """
    return cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap) 

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Extrapolate lines

In [3]:
def extrapolate_lines(raw_lines, num_lines, use_lr=True, use_km=True):    
    """Extrapolate lines by clustering and linear regression.
    The clustering is done by KMeans, which requires a prior knowledge of the number cluster.
    In this case, we know there are two: left lane and right lane.
    Additional parameter include whether use Linear Regression or simple averaging,
    and whether Kmeans for clustering.
    """
    # Convert each line segment from (x1, y1, x2, y2) to (slope, intercept) space.
    # For numerical stability, add a small epilson to handle the case of vertical lines.
    # To improve, we could convert to (rho, theta) space where all lines are well defined.
    points = list()
    for line in raw_lines:
        for x1, y1, x2, y2 in line:
            points.append([(y1-y2) / (x1-x2+1e-6), (y2*x1-y1*x2) / (x1-x2+1e-6), abs(x1-x2)+1e-6])      
    points = np.array(points)
    #print(points[:,0])
    #points_normalized = normalize(points)
    #print(point_normalized)

    if use_km:
        # Run KMeans clustering based on the Eclidean distance in Hough space.
        # There is a lot of variance in intercept. Slope, on the other hand, is more stable.
        # So, just feed slope for KMeans clustering.
        labels = KMeans(n_clusters=num_lines, random_state=0).fit(points[:,0].reshape(-1,1)).labels_
    else:
        # KMeans is not stable in the case where there is a line with large slope.
        labels = np.array([1 if s >=0 else 0 for s, _, _ in points])
    #print(labels)


    # Find each line (left, right)
    learned_lines = list()
    for l in range(num_lines):
        #print('For line %d' % l)
        #print(raw_lines[labels==l])
        mass_center = np.average(points[labels==l], axis=0)
        #print(mass_center)
        if use_lr:
            # Run a linear regression to get the average slope and intercept
            x = list()
            y = list()
            for line in raw_lines[labels==l]:
                for x1,y1,x2,y2 in line:
                    x.extend([x1, x2])
                    y.extend([y1, y2])
            #print(x)
            #print(y)
            # Add per-sample weights based on the Eclidean distance to the mass center.
            # Otherwise, the regression is overly sensitive to outliers (due to mis-classification by KMeans)
            # Extreme slope lines are tricky to deal: it can be easily mis-clustered.
            # This is a bit hacky: just throw it away.
            # Due to the high variance of intercept, we just use it to generate weights.
            weights = np.array([x/math.sqrt((s - mass_center[0])**2 + (b - mass_center[1])**2)
                                if 5 >= abs(s) >= 0.5 else 0
                                for s, b, x in points[labels==l]])
            # Clip the weights to avoid being too large.
            #print(weights)
            #weights[weights > 300] = 300
            # Each weight corresponds to two x-y points.
            weights2 = list()
            for w in weights:
                weights2.extend([w, w])
            # Use X to adjust weights, since the two lines are easily separable in X axis.
            x_std = np.std(x)
            x_avg = np.average(x)
            weights2 = np.array(weights2)
            #print(weights2)
            weights2[abs(x - x_avg) > 1.5 * x_std] = 0
            if np.all(weights2==0):
                # this is a degenerated case. Set weight to non-zero otherwise the fit will fail.
                weights2=np.random.uniform(size=weights2.shape)
            #print(weights2)
            # Normalize the weights
            weights_n = normalize([weights2])[0,:]
            #print(weights_n)
            lr = linear_model.LinearRegression()
            lr.fit(np.array(x).reshape(-1,1), y, weights_n)
            learned_lines.append([lr.coef_[0], lr.intercept_])
        else:
            # Simple averaging
            learned_lines.append(np.average(points[labels==l], axis=0))
        
    # Return the KMeans labels for debugging.
    #print(learned_lines)
    return learned_lines, labels

## Parameters for tunning

In [4]:
# The region is a ratio for the input image frame.
lane_region = [
    (120.0/960, 1.0),
    (430.0/960, 330.0/540),
    (540.0/960, 330.0/540),
    (950.0/960, 1.0)
]
# Parameters for rough transformation
rho = 2              # distance resolution in pixels of the Hough grid
theta = np.pi/180    # angular resolution in radians of the Hough grid
threshold = 10       # minimum number of votes (intersections in Hough grid cell)
min_line_length = 20 # minimum number of pixels making up a line
max_line_gap = 40    # maximum gap in pixels between connectable line segments

### Debugging session

In [ ]:
def debug_find_lane_lines(raw_image, lane_region, rho, theta, threshold, min_line_length, max_line_gap, extrapolate, use_lr, use_km):
    """Find the lanes and draw them on an empty image.
    """
    # Convert image to gray scale
    grey_image = grayscale(raw_image)
    # Apply Gaussian blur
    blur_image = gaussian_blur(grey_image, 3)
    # Detect edges using Canny transformation
    edges = canny(blur_image, 50, 150)
    # Crop according to the area of interest
    region = [(x * edges.shape[1], y * edges.shape[0]) for x, y in lane_region]
    masked_edges = region_of_interest(edges, region)
    # Find lines using Hough transformation
    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    plt.figure()
    plt.imshow(masked_edges)
    
    lanes_image = np.zeros_like(raw_image)
    if extrapolate:
        # Extrapolate to find left and right lanes
        def y(x, param): return int(round(x*param[0] + param[1]))
        lp, labels = extrapolate_lines(lines, 2, use_lr, use_km)
        lanes = [[0, y(0, p), raw_image.shape[1], y(raw_image.shape[1], p)]
                 for p in lp]
        #print(lp)
        # Draw the lanes full extended
        #draw_lines(lanes_image, np.array([lanes]), thickness=5)
        #draw_lines(lanes_image, lines[labels==0], color=[0,0,255])
        draw_lines(lanes_image, lines[labels==1], color=[0,255,0])
        # Last, bound the lines in the area of interest
        lanes_image = region_of_interest(lanes_image, region)
    else:
        draw_lines(lanes_image, lines, thickness=5)
    return lanes_image

raw_image = mpimg.imread('debug_images/debug-003.jpg')
lanes = debug_find_lane_lines(raw_image, lane_region, 
                              rho, theta, threshold, min_line_length, max_line_gap,
                              extrapolate=True, use_lr=True, use_km=False)
# Overlay the lanes on original image
annotated = weighted_img(lanes, raw_image, α=0.1)
plt.figure()
plt.imshow(annotated)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [5]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

def find_lane_lines(raw_image, lane_region, rho, theta, threshold, min_line_length, max_line_gap,
                    extrapolate, use_lr, use_km):
    """Find the lanes and draw them on an empty image.
    """
    # Convert image to gray scale
    grey_image = grayscale(raw_image)
    # Apply Gaussian blur
    blur_image = gaussian_blur(grey_image, 3)
    # Detect edges using Canny transformation
    edges = canny(blur_image, 50, 150)
    # Crop according to the area of interest
    region = [(x * edges.shape[1], y * edges.shape[0]) for x, y in lane_region]
    masked_edges = region_of_interest(edges, region)
    # Find lines using Hough transformation
    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    lanes_image = np.zeros_like(raw_image)
    if extrapolate:
        # Extrapolate to find left and right lanes
        def y(x, param): return int(round(x*param[0] + param[1]))
        lp, labels = extrapolate_lines(lines, 2, use_lr, use_km)
        lanes = [[0, y(0, p), raw_image.shape[1], y(raw_image.shape[1], p)]
                 for p in lp]
        # Draw the lanes full extended
        draw_lines(lanes_image, np.array([lanes]), thickness=5)
        # Last, bound the lines in the area of interest
        lanes_image = region_of_interest(lanes_image, region)
    else:
        draw_lines(lanes_image, lines, thickness=5)
    return lanes_image


img_folder = 'test_images'
out_folder = 'test_images_output'
for f in os.listdir(img_folder):
    if not f.endswith('.jpg'):
        continue
    # Read in image from file system
    raw_image = mpimg.imread(os.path.join(img_folder, f))
    # Find lanes.
    lanes = find_lane_lines(raw_image, lane_region, 
                            rho, theta, threshold, min_line_length, max_line_gap,
                            extrapolate=True, use_lr=True, use_km=False)
    # Overlay the lanes on original image
    annotated = weighted_img(lanes, raw_image)
    #plt.figure()
    #plt.imshow(annotated)
    # Write back to file system
    print("wrting back " + os.path.join(out_folder, f))
    cv2.imwrite(os.path.join(out_folder, f), cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))
    

wrting back test_images_output/solidWhiteCurve.jpg
wrting back test_images_output/solidWhiteRight.jpg
wrting back test_images_output/solidYellowCurve.jpg
wrting back test_images_output/solidYellowCurve2.jpg
wrting back test_images_output/solidYellowLeft.jpg
wrting back test_images_output/whiteCarLaneSwitch.jpg


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [9]:
idx = 1

def process_image(raw_image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    global idx
    filename = 'debug_images/debug-%03d.jpg' % idx
    cv2.imwrite(filename, cv2.cvtColor(raw_image, cv2.COLOR_RGB2BGR))
    raw_image = mpimg.imread(filename)
    # Find lanes.
    lanes = find_lane_lines(raw_image, lane_region, 
                            rho, theta, threshold, min_line_length, max_line_gap,
                            extrapolate=True, use_lr=True, use_km=False)
    # Overlay the lanes on original image
    overlaid =  weighted_img(lanes, raw_image)
    #cv2.imwrite('debug_images_output/debug-%03d.jpg' % idx, cv2.cvtColor(overlaid, cv2.COLOR_RGB2BGR))
    
    idx+=1
    return overlaid

Let's try the one with the solid white lane on the right first ...

In [10]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,2)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:15<00:00, 14.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 9.53 s, sys: 1.02 s, total: 10.6 s
Wall time: 16 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [12]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(9,11)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:48<00:00, 14.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 30 s, sys: 2.84 s, total: 32.8 s
Wall time: 49.4 s


In [14]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [15]:
lane_region = [
    (120.0/960, 0.9),
    (430.0/960, 330.0/540),
    (540.0/960, 330.0/540),
    (950.0/960, 0.9)
]

challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(4,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:33<00:00,  8.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 20.1 s, sys: 2.08 s, total: 22.2 s
Wall time: 35.5 s


In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))